In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50V2
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

def create_classifier(input_shape=(32, 32, 3), num_classes=10):
    base_model = ResNet50V2(include_top=False, weights="imagenet", input_shape=input_shape, pooling="avg")
    x = base_model.output
    x = layers.Dense(512, activation='relu')(x)
    predictions = layers.Dense(num_classes, activation='softmax')(x)
    classifier = models.Model(inputs=base_model.input, outputs=predictions)
    classifier.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return classifier

classifier = create_classifier()
classifier.summary()

# Initial training
num_initial_samples = 2500
classifier.fit(x_train[:num_initial_samples], y_train[:num_initial_samples], epochs=1)

score = classifier.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


# Active learning
num_iterations = 10
num_queries = 250

x_train_unlabeled = x_train[num_initial_samples:]
y_train_unlabeled = y_train[num_initial_samples:]

for iteration in range(num_iterations):
    # Compute entropy for each sample
    probabilities = classifier.predict(x_train_unlabeled)
    entropy = -np.sum(probabilities * np.log2(probabilities + 1e-9), axis=1)
    
    # Select samples with highest entropy
    query_indices = np.argsort(entropy)[-num_queries:]
    x_query = x_train_unlabeled[query_indices]
    y_query = y_train_unlabeled[query_indices]
    
    # Remove queried samples from the unlabeled pool
    x_train_unlabeled = np.delete(x_train_unlabeled, query_indices, axis=0)
    y_train_unlabeled = np.delete(y_train_unlabeled, query_indices, axis=0)
    
    # Add queried samples to the training dataset
    x_train = np.concatenate([x_train, x_query], axis=0)
    y_train = np.concatenate([y_train, y_query], axis=0)
    
    # Train the classifier on the updated training dataset
    classifier.fit(x_train, y_train, epochs=1)
    
    score = classifier.evaluate(x_test, y_test, verbose=0)
    print("Test iter:", iteration)
    print("Test accuracy:", score[1])


# Evaluate the classifier
score = classifier.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 18, 18, 64)   0           ['conv1_conv[0][0]']             
                                                                                              

1594/1594 [==============================] - 245s 154ms/step - loss: 1.7167 - accuracy: 0.3497
Test iter: 0
Test accuracy: 0.2379000037908554


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

11490434/11490434 [==============================] - 2s 0us/step


(18000, 32, 32, 3)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50V2
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

def create_classifier(input_shape=(32, 32, 3), num_classes=10):
    base_model = ResNet50V2(include_top=False, weights="imagenet", input_shape=input_shape, pooling="avg")
    x = base_model.output
    x = layers.Dense(512, activation='relu')(x)
    predictions = layers.Dense(num_classes, activation='softmax')(x)
    classifier = models.Model(inputs=base_model.input, outputs=predictions)
    classifier.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return classifier

classifier = create_classifier()
classifier.summary()

# Initial training
num_initial_samples = 2500
classifier.fit(x_train[:num_initial_samples], y_train[:num_initial_samples], epochs=1)

score = classifier.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Active learning
num_iterations = 10
num_queries = 250

x_train_unlabeled = x_train[num_initial_samples:]
y_train_unlabeled = y_train[num_initial_samples:]

for iteration in range(num_iterations):
    # Compute confidence for each sample
    probabilities = classifier.predict(x_train_unlabeled)
    confidence = np.max(probabilities, axis=1)

    # Select samples with lowest confidence
    query_indices = np.argsort(confidence)[:num_queries]
    x_query = x_train_unlabeled[query_indices]
    y_query = y_train_unlabeled[query_indices]

    # Remove queried samples from the unlabeled pool
    x_train_unlabeled = np.delete(x_train_unlabeled, query_indices, axis=0)
    y_train_unlabeled = np.delete(y_train_unlabeled, query_indices, axis=0)

    # Add queried samples to the training dataset
    x_train = np.concatenate([x_train, x_query], axis=0)
    y_train = np.concatenate([y_train, y_query], axis=0)

    # Train the classifier on the updated training dataset
    classifier.fit(x_train, y_train, epochs=1)
    
    score = classifier.evaluate(x_test, y_test, verbose=0)
    print("Test iter:", iteration)
    print("Test accuracy:", score[1])

# Evaluate the classifier
score = classifier.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


10000

(48000, 28, 28)